In [1]:
## Importación al bucket iedatalakeLanding de entidades de Microsoft Dynamics AX
## ------------------------------------------------------------------------------
##
## Las entidades necesarias se extraerán de la base de datos de AX. 

import sqlalchemy
import csv
import string
import pandas as pd
import os
import shutil
import datetime
import uuid
import sys

# Importación de opciones de configuración

if (os.name == 'nt'):
    sys.path.append(os.environ["USERPROFILE"].replace('\\', '/') + '/iedatalake/python/config')
else: 
    sys.path.append('/home/ubuntu/iedatalake/python/config')
import databaseconfig as dbc

In [2]:
## Proceso, versíón y fecha de ingesta para identificar la ejecución

ingestionProcess = 'Ingesta de Entidades AX desde analytics01 en python'
ingestionVersion = str(uuid.uuid1())
ingestionInitialDate = str(datetime.datetime.now())

## Motor SQLAlchemy para la base de datos SQL Server de AX

axEngine = sqlalchemy.create_engine(dbc.ax['dialect'] + '+' + dbc.ax['driver'] + '://' \
      + dbc.ax['username'] + ':' + dbc.ax['password'] + '@' \
      + dbc.ax['host'] + '/' + dbc.ax['database'] + dbc.ax['parameters'])

## Carpeta auxiliar para depositar los CSVs generados antes de subir

if not os.path.exists("data"):
    os.makedirs("data")


In [4]:
## Procedimiento de importación de una entidad de AX a un fichero CSV local

def ImportAXEntity(entityName, csvIndex, **optionalParameters): 
    
    # Ruta del fichero CSV, asumida en una subcarpeta "data" en la carpeta donde se ejecuta
    # y extensión .csv.gz porque se generará comprimido en formato GZIP
    
    csvFile = optionalParameters.get('csvFile', 'data/' + entityName + '.csv.gz')
        
    # Parámetro opcional con el número de registros del bloque a recuperar de forma iterativa
    
    chunkSize = optionalParameters.get('chunkSize', None)
    if (chunkSize == None):
        # Si no viene de entrada ningún número de registros, no se limita, poniendo como alcance
        # el número total de filas de la tabla de la entidad original
        axConnection = axEngine.connect()
        df = pd.io.sql.read_sql("SELECT COUNT(*) as recuento FROM " + entityName, axConnection)
        chunkSize = int(df["recuento"]) + 1
        axConnection.close()
    else: 
        chunkSize = int(chunkSize)

    # Convertimos csvIndex a lista, pues llega como una cadena en el parámetro 
    # con valores separados por coma de la forma 'DATAAREAID, DIMENSIONCODE, NUM' 
    # dejando el valor original en orderBy para la ordenación en la consulta fraccionada
    
    orderBy = csvIndex    
    csvIndex = csvIndex.replace(' ', '').split(',')    
    axConnection = axEngine.connect()
    rowsPending = True
    offset = 0
    while rowsPending:
        sqlSentence = "WITH Results_SQL AS (SELECT "\
        + "ROW_NUMBER() OVER " \
        + "(ORDER BY " + orderBy + ") as RowNum, * " \
        + "FROM " + entityName + ") " \
        + "SELECT * FROM Results_SQL WHERE  RowNum > " + str(offset) + " AND RowNum <=  " \
        + str(offset + chunkSize)        
        chunk = pd.io.sql.read_sql(sqlSentence, axConnection)

        print (str(offset)  + " < RowNum <= " + str(offset + chunkSize))

        # Borramos columna auxiliar RowNum usada para ordenar por el índice
        del chunk["RowNum"]
        chunk.set_index(csvIndex, inplace = True)        
        if (offset == 0):
            chunk.to_csv(csvFile, sep = ';', compression = 'gzip', quotechar='|', quoting=csv.QUOTE_MINIMAL, 
                     encoding = 'utf8')
        else:            
            chunk.to_csv(csvFile, sep = ';', compression = 'gzip', quotechar='|', quoting=csv.QUOTE_MINIMAL, 
                     encoding = 'utf8', mode= 'a', header = False)               
        offset += chunkSize
        if (len(chunk) < chunkSize):
            rowsPending = False            
        del chunk

    axConnection.close()  
    


In [5]:
ImportAXEntity(entityName = 'LEDGERTABLE', csvIndex = 'DATAAREAID, ACCOUNTNUM')

0 < RowNum <= 3677


In [11]:
ImportAXEntity(entityName = 'DIMENSIONS', csvIndex = 'DATAAREAID, DIMENSIONCODE, NUM')

0 < RowNum <= 180767


In [6]:
ImportAXEntity(entityName = 'LEDGERTRANS', csvIndex = 'DATAAREAID, RECID', chunkSize = 300000)

0 < RowNum <= 300000
300000 < RowNum <= 600000
600000 < RowNum <= 900000
900000 < RowNum <= 1200000
1200000 < RowNum <= 1500000
1500000 < RowNum <= 1800000
1800000 < RowNum <= 2100000
2100000 < RowNum <= 2400000
2400000 < RowNum <= 2700000
2700000 < RowNum <= 3000000
3000000 < RowNum <= 3300000
3300000 < RowNum <= 3600000
3600000 < RowNum <= 3900000
3900000 < RowNum <= 4200000
4200000 < RowNum <= 4500000
4500000 < RowNum <= 4800000
4800000 < RowNum <= 5100000
5100000 < RowNum <= 5400000
5400000 < RowNum <= 5700000
5700000 < RowNum <= 6000000
6000000 < RowNum <= 6300000
6300000 < RowNum <= 6600000
6600000 < RowNum <= 6900000
6900000 < RowNum <= 7200000
7200000 < RowNum <= 7500000
7500000 < RowNum <= 7800000
7800000 < RowNum <= 8100000
8100000 < RowNum <= 8400000
8400000 < RowNum <= 8700000


In [12]:
## Sincronización de carpeta local "data" con carpeta "ax" del bucket S3 iedatalakelanding
## Hace uso de AWSCLI (previamente instalado mediante la instrucción: sudo apt install awscli)

ingestionFinalDate = str(datetime.datetime.now())
os.system('aws s3 sync data s3://iedatalakelanding/ax --metadata ingestionprocess="' + ingestionProcess  
          + '",ingestionversion="' + ingestionVersion + '",ingestioninitialdate="' + ingestionInitialDate 
          + '",ingestionfinaldate="' + ingestionFinalDate + '",code="Ingest-AX-Entities"');